<a href="https://colab.research.google.com/github/n00bminion/BrainTeaser/blob/main/Black_Jack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import pandas as pd

class Card():
    def __init__(self, value, suit):
        self.value = value
        self.suit = suit
        self._mapping  = {
            ('J', 'Q', 'K'): 10,
            ('A'): 1
        }
        self.int_value = self._convertValue(value)

    def _convertValue(self, value):
        if isinstance(value, int):
            return value
        elif isinstance(value, str):
            for key in self._mapping:
                if value in key:
                    return self._mapping.get(key)

    def __eq__(self, obj: object) -> bool:
        return self.int_value == obj.int_value

    def __str__(self) -> str:
        return f'{self.value}{self.suit}'

    def __repr__(self) -> str:
        return f'{self.value}{self.suit}'

class CardDeck():
    def __init__(
            self,
            suits = ["\u2663", "\u2665", "\u2666", "\u2660"],
            values = ['A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K']
        ):

        self.suits = suits
        self.values = values
        self.deck = [Card(value, suit) for suit in self.suits for value in self.values]
        random.shuffle(self.deck)

    @property
    def allCards(self):
        return self.deck

class Dealer():
    def __init__(self, min_no_of_decks = 4, max_no_of_decks = 8):
        self.no_of_decks = random.randint(min_no_of_decks, max_no_of_decks)
        self.cards = [card for deck in range(0,self.no_of_decks) for card in CardDeck().allCards]
        self.hand = Hand()
        self.payout_multiplier = {"black jack":1.5,"normal":1}
        self.total_limit = 17

    def reShuffleCards(self):
        self.cards = [card for deck in range(0,self.no_of_decks) for card in CardDeck().allCards]
        return self

    def __str__(self):
        _line = '-'*100
        _str = f'''Dealer\nHand:\t[{self.hand}]\n{_line}'''
        return _str

    def __repr__(self):
        _line = '-'*100
        _str = f'''Dealer\nHand:\t[{self.hand}]\n{_line}'''
        return _str

    def deal(self, player, hand):
        player.hit(self.cards[0], hand)
        self.cards.pop(0)
        return self

    def selfDeal(self):
        self.hand =  self.hand + self.cards[0]
        self.cards.pop(0)
        return self

    def clearHand(self):
        self.hand = Hand()
        return self

class Player():
    def __init__(
            self,
            name,
            balance=10_000
        ):
        self.name = name
        self.balance = balance
        self.hands = []
        self.action = None
        self.current_hand = None
        self.consecutive_wins = 0

    @property
    def _str(self):
        _hand = "\n\t"+"\n\t".join(map(str, self.hands)) + "\n\t"
        _line = '-'*100
        _str = f'''\nPlayer: {self.name}\nBalance: {self.balance:,}\nCurrent Hand: [\n\t{self.current_hand}\n\t]\nHand(s): [{_hand}]\nAction: [{self.action}]\n{_line}\n'''
        return _str

    def __str__(self):
        return self._str

    def __repr__(self):
        return self._str

    def bet(self, bet_amount, create_new_hand=True):
        self.balance = self.balance - bet_amount
        if create_new_hand:
            self.hands.append(
                Hand(bet_amount=bet_amount)
                )
        return self

    def hit(self, card, hand):
        hand = hand + card ##__add__
        return self

    def stand(self, hand):
        hand.active_state = False
        return self

    def doubleDown(self, card, hand):
        hand.bet_amount += hand.bet_amount
        self.hit(card, hand)
        hand.active_state = False
        return self

    def split(self, hand):
        c1, c2 = hand.cards
        h1, h2 = Hand(bet_amount = hand.bet_amount, cards = c1), Hand(bet_amount = hand.bet_amount, cards = c2)
        self.hands.pop(-1)
        self.hands.append(h1)
        self.hands.append(h2)
        self.bet(hand.bet_amount, create_new_hand = False)
        return self

    def clearHand(self):
        self.hands = []
        self.action = None
        self.current_hand = None
        return self

class Hand():
    def __init__(
            self,
            bet_amount = 0,
            cards = None
            ):
        if not cards:
            self.cards = []
        elif isinstance(cards,list):
            self.cards = cards
        elif isinstance(cards, Card):
            self.cards = [cards]

        self.bet_amount = bet_amount
        self.active_state = True
        self._is_black_jack = False
        self._total = 0
        self._hand_type = None
        self.state = None

    @property
    def total(self):
        return sum([card.int_value for card in self.cards])

    @total.setter
    def total(self, value):
        self._total = value
        return self._total

    @property
    def number_of_cards(self):
        return len(self.cards)

    @property
    def is_black_jack(self):
        if (len(self.cards)==2) and ('A' in [c.value for c in self.cards]) and (self.total == 11):
            return True
        else:
            return False

    @property
    def hand_type(self):
        if self.is_black_jack:
            self.active_state = False
            self._hand_type = 'black jack'
            return self._hand_type

        if (not self.cards) or (len(self.cards)==1):
            return self._hand_type

        if all(card.value ==self.cards[0].value for card in self.cards):
            self._hand_type = 'pair'
        elif not all(card.value ==self.cards[0].value for card in self.cards):
            if ('A' in [card.value for card in self.cards]) and (len(self.cards)==2):
                self._hand_type = 'soft total'
            else:
                self._hand_type = 'hard total'

        return self._hand_type

    def __add__(self, card):
        self.cards.append(card)
        return self

    def __contains__(self, card):
        return card.value in [card.value for card in self.cards]

    def __str__(self):
        _str = f'''State: {'Active' if self.active_state else 'Inactive'}, Cards: {self.cards}, Total Count: {self.total}, Bet Amount = {self.bet_amount}, No. Of Cards: {self.number_of_cards}, Hand Type: {self.hand_type}'''
        return _str

    def __repr__(self):
        _str = f'''State: {'Active' if self.active_state else 'Inactive'}, Cards: {self.cards}, Total Count: {self.total}, Bet Amount = {self.bet_amount}, No. Of Cards: {self.number_of_cards}, Hand Type: {self.hand_type}'''
        return _str

class BasicStrategy():
    def __init__(self):
        wiki_bs = pd.read_html('https://en.wikipedia.org/wiki/Blackjack')[3]
        wiki_bs.iloc[0] = wiki_bs\
            .iloc[0]\
            .shift(periods=1, axis=0)
        wiki_bs.columns = [str(i) for i in wiki_bs.loc[0]]
        wiki_bs.rename({'None':'value'}, axis = 1, inplace = True)
        hard_total, soft_total, pair = (
            wiki_bs.loc[2:11].copy(),
            wiki_bs.loc[14:19].copy(),
            wiki_bs.loc[22:30].copy()
            )

        hard_total.value = hard_total.value\
            .apply(
                lambda x: int(x)
                if x.isdigit()
                else list(
                    range(
                        int(x.split('–')[0]),
                        1+int(x.split('–')[1])
                        )
                    )
                )
        hard_total = hard_total.explode('value')
        hard_total.index = hard_total.value\
            .sort_values(ascending=False)
        for col in ['J','Q','K']:
            hard_total[col] = hard_total['10']
        self.hard_total = hard_total\
            .drop(['value'], axis=1)\
            .replace({'Us':'S', 'Uh':'H', 'Dh':'D'})

        soft_total.value = soft_total.value\
            .apply(
                lambda x: [x.split(',')]
                if '–' not in x
                else [[i] for i in x.split('–')]
                )
        soft_total = soft_total.explode('value')
        soft_total.index = soft_total.value.apply(lambda x: tuple(x[0].split(',')) if len(x[0]) > 1 else tuple(x))
        for col in ['J','Q','K']:
            soft_total[col] = soft_total['10']
        self.soft_total = soft_total\
            .replace({'Ds':'D', 'Dh':'D'})\
            .drop(['value'], axis=1)

        pair.value = pair.value\
            .apply(lambda x: [[x[0:int(len(x)/2)] ,x[0:int(len(x)/2)] ]]
                if x.isdigit()
                else [[i] for i in x.split('–') ]
                if '–' in x
                else [x.split(',')]
                )

        pair = pair.explode('value')
        pair.index = pair.value.apply(lambda x: tuple(x[0].split(',')) if len(x[0]) > 2 else tuple(i.strip() for i in x))
        for col in ['J','Q','K']:
            pair[col] = pair['10']
        self.pair = pair.drop(['value'], axis=1)\
            .replace({'Usp':'SP', 'Dh':'D'})

    def __call__(
            self,
            hand,
            dealer_card
            ):

        player_cards =  [card for card in hand.cards if 'A' in str(card.value)]\
                        + [card for card in hand.cards if 'A' not in str(card.value)]

        if 'A' in [card.value for card in hand.cards]:
            upper = hand.total + 10
            lower = hand.total

            if upper <= 21:
                hand.total = upper

        if hand.hand_type == 'pair':
            return self.pair\
                [str(dealer_card.value)]\
                [
                    (
                        str(player_cards[0].value) if str(player_cards[0].value) not in ['J','Q','K'] else '10',
                        str(player_cards[1].value) if str(player_cards[1].value) not in ['J','Q','K'] else '10'
                    )
                ]

        elif hand.hand_type == 'soft total':
            return self.soft_total\
                [str(dealer_card.value)]\
                [(str(player_cards[0].value),str(player_cards[1].value))]

        elif hand.hand_type == 'hard total':
            try:
                return self.hard_total\
                    [str(dealer_card.value)]\
                    [hand.total]
            except Exception as e:
                print('hand: ', hand)
                print('dealers card: ', dealer_card)
                raise e

## action depending on the basic strategy signal
def _action(
        player_choice_signal,
        player,
        hand,
        dealer
        ):
    if player_choice_signal == 'S':
        player.stand(hand)
    elif player_choice_signal == 'H':
        dealer.deal(player,hand)
    elif player_choice_signal == 'D':
        player.doubleDown(dealer.cards[0], hand)
        dealer.cards.pop(0)
        hand._active_state = False
    elif player_choice_signal == 'SP':
        player.split(hand)

## decide the winner
def _winner(
    hand,
    dealer
):

    ## guarantee lost
    if (hand.total > 21) or (dealer.hand.hand_type == 'black jack'):
        winner = 'dealer'

    ## checking hand
    ## if blackjack
    elif hand.hand_type == 'black jack':
        winner = 'player' if (dealer.hand.hand_type != 'black jack')\
        or (dealer.hand.total > 21)\
        else 'draw'

    ## not black jack but still is 21 total
    elif hand.total == 21:
        if (dealer.hand.total != 21):
            winner = 'player'

    elif hand.total < 21:
        if hand.total > dealer.hand.total:
            winner = 'player' if (dealer.hand.total <= 21) else 'dealer'
        elif hand.total < dealer.hand.total:
            winner = 'dealer' if (dealer.hand.total <= 21) else 'player'

    if (hand.total == dealer.hand.total) and (hand.total<=21):
        winner = 'draw'

    return winner


basic_strategy = BasicStrategy()

## black jack rules:
##      player get 2 cards at the start whilst dealer only get 1
##      player hit until they get close to or at 21 based on basic strategy
##      "A" counts as 1 initially unless it's black jack or close to 21
##      player will split based on basic strategy
##      player will double down based on basic strategy
##      we will keep a running card count and a true running card count to evaluate its effectiveness
##      we will keep a stats display of probably of each card and the next best card

def play(
    players: list[Player],
    no_of_rounds: int = 10,
    cut_size: float = 0.5,## 50%
    base_bet_amount = 10
    #display_statistics: bool = True
):
    dealer = Dealer()
    cut_size = round(len(dealer.cards)*cut_size,0)
    base_bet_amount = 10
    df = pd.DataFrame(
        columns=[
            'Round',
            'PlayerName',
            'PlayerStartBalance',
            'PlayerEndBalance',
            'PlayerHand',
            'PlayerHandTotal',
            'PlayerHandBetAmount',
            'PlayerHandType',
            'DealerHand',
            'DealerHandTotal',
            'Winner'
        ]
    )

    if no_of_rounds:
        r = 0

    ## each round
    while True:

        if no_of_rounds:
            r=r+1

        ## initial deal, 2 cards each for player and 1 card for dealer
        for player in players:
            starting_balance = player.balance
            #print(f"player {player.name}'s starting balance: ", player.balance)
            player.bet(base_bet_amount * (2**player.consecutive_wins))
            for _ in range(2):
                dealer.deal(player, player.hands[0])

        dealer.selfDeal()
        #print(dealer)

        ## each player take turn playing
        for player in players:
            i = 0

            ## iterate over each hand (can be more than 1 hand)
            while True:
                hand = player.hands[i]
                player.current_hand = hand

                # play each hand depending on the basic strategy
                while True:
                    #print(player)
                    if not hand.active_state:
                        break

                    if hand.number_of_cards == 1:
                        dealer.deal(player, hand)
                        #print(player)

                    choice = basic_strategy(hand,dealer.hand.cards[0])
                    _action(choice, player, hand, dealer)
                    player.action = choice

                    if player.action == 'SP':
                        break

                    if hand.total > 21:
                        player.stand(hand)

                if len(player.hands) == i+1:
                    break

                ## loop over the same hand again (i.e. not +1 to index)  when split
                if player.action == 'SP':
                    continue
                else:
                    i+=1


        ## when all players finish playing, dealer plays last
        while True:
            dealer.selfDeal()

            ## dealer stop if larger than or eq to 17
            if dealer.hand.total >= dealer.total_limit:
                break

        #print(dealer)

        ## finally we assess hand(s) of players against dealer
        for player in players:
            for hand in player.hands:
                winner = _winner(hand, dealer)

                if (winner == 'player'):
                    if (hand.hand_type == 'black jack'):
                        player.balance = player.balance + hand.bet_amount + (hand.bet_amount * dealer.payout_multiplier['black jack'])
                    else:
                        player.balance = player.balance + hand.bet_amount + (hand.bet_amount * dealer.payout_multiplier['normal'])

                    if player.consecutive_wins == 3:
                        player.consecutive_wins = 0
                    else:
                        player.consecutive_wins = player.consecutive_wins + 1

                elif (winner == 'draw'):
                    player.balance = player.balance + hand.bet_amount
                    player.consecutive_wins = 0

                elif (winner == 'dealer'):
                    player.consecutive_wins = 0

                if r == no_of_rounds:
                    df = pd.concat(
                        [
                            df,
                            pd.DataFrame.from_dict({
                                'Round':r,
                                'PlayerName':player.name,
                                'PlayerStartBalance':starting_balance,
                                'PlayerEndBalance':player.balance,
                                'PlayerHand':hand.cards,
                                'PlayerHandTotal':hand.total,
                                'PlayerHandBetAmount':hand.bet_amount,
                                'PlayerHandType':hand.hand_type,
                                'DealerHand':dealer.hand.cards,
                                'DealerHandTotal':dealer.hand.total,
                                'Winner':winner
                            }, orient='index').T
                    ],
                        axis=0
                    )

                # print(f"player {player.name}'s consecutive wins: ", player.consecutive_wins)
                # print(f"player {player.name}'s final balance: ", player.balance, flush=True)
                # print(f"player {player.name}'s hand: ", hand, flush=True)
                # print(f"dealer's hand: ", dealer.hand, flush=True)
                # print('winner:', winner, end = '\n\n', flush=True)

            player.clearHand()

        dealer.clearHand()

        ## at around 50% of dealer's cards, the cards are reshuffled
        if len(dealer.cards) <= cut_size:
            dealer.reShuffleCards()

        if r == no_of_rounds:
            break

    return df

df = pd.DataFrame()

for i in range(1, 51):
    print(i)
    _df = play(
        players=[
            Player(name='1', balance = 1000),
            Player(name='2', balance = 1000),
            Player(name='3', balance = 1000),
            Player(name='4', balance = 1000),
            Player(name='5', balance = 1000)
            ],
        no_of_rounds = 20
        )

    df = pd.concat([df, _df])



1
2
hand:  State: Active, Cards: [2♠, A♥, A♣], Total Count: 4, Bet Amount = 40, No. Of Cards: 3, Hand Type: hard total
dealers card:  3♦


KeyError: ignored

In [ ]:
BasicStrategy().soft_total

,2,3,4,5,6,7,8,9,10,A,J,Q,K
value,,,,,,,,,,,,,
"(A, 9)",S,S,S,S,S,S,S,S,S,S,S,S,S
"(A, 8)",S,S,S,S,D,S,S,S,S,S,S,S,S
"(A, 7)",D,D,D,D,D,S,S,H,H,H,H,H,H
"(A, 6)",H,D,D,D,D,H,H,H,H,H,H,H,H
"(A, 4)",H,H,D,D,D,H,H,H,H,H,H,H,H
"(A, 5)",H,H,D,D,D,H,H,H,H,H,H,H,H
"(A, 2)",H,H,H,D,D,H,H,H,H,H,H,H,H
"(A, 3)",H,H,H,D,D,H,H,H,H,H,H,H,H


In [ ]:
BasicStrategy().pair

,2,3,4,5,6,7,8,9,10,A,J,Q,K
value,,,,,,,,,,,,,
"(A, A)",SP,SP,SP,SP,SP,SP,SP,SP,SP,SP,SP,SP,SP
"(10, 10)",S,S,S,S,S,S,S,S,S,S,S,S,S
"(9, 9)",SP,SP,SP,SP,SP,S,SP,SP,S,S,S,S,S
"(8, 8)",SP,SP,SP,SP,SP,SP,SP,SP,SP,SP,SP,SP,SP
"(7, 7)",SP,SP,SP,SP,SP,SP,H,H,H,H,H,H,H
"(6, 6)",SP,SP,SP,SP,SP,H,H,H,H,H,H,H,H
"(5, 5)",D,D,D,D,D,D,D,D,H,H,H,H,H
"(4, 4)",H,H,H,SP,SP,H,H,H,H,H,H,H,H
"(2, 2)",SP,SP,SP,SP,SP,SP,H,H,H,H,H,H,H


In [ ]:
BasicStrategy().hard_total

,2,3,4,5,6,7,8,9,10,A,J,Q,K
value,,,,,,,,,,,,,
21,S,S,S,S,S,S,S,S,S,S,S,S,S
20,S,S,S,S,S,S,S,S,S,S,S,S,S
19,S,S,S,S,S,S,S,S,S,S,S,S,S
18,S,S,S,S,S,S,S,S,S,S,S,S,S
17,S,S,S,S,S,S,S,S,S,S,S,S,S
16,S,S,S,S,S,H,H,H,H,H,H,H,H
15,S,S,S,S,S,H,H,H,H,H,H,H,H
14,S,S,S,S,S,H,H,H,H,H,H,H,H
13,S,S,S,S,S,H,H,H,H,H,H,H,H
